In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from multiprocess import Pool
import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

from registration_utilities import *

/home/yuncong/MouseBrainAtlas/src/utilities/utilities2015.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7

Setting environment for Precision WorkStation
{'DEMO998': 0.46}


In [2]:
# For DEMO998, use atlasV7
atlas_spec = dict(name='atlasV7',
                   vol_type='score'    ,               
                    resolution='10.0um'
                   )

# atlas_spec = dict(name='atlasV6',
#                    vol_type='score'    ,               
#                     resolution='10.0um'
#                    )

In [3]:
atlas_structures_wrt_canonicalAtlasSpace_atlasResol = \
DataManager.load_original_volume_all_known_structures_v3(atlas_spec, in_bbox_wrt='canonicalAtlasSpace',
                                                        out_bbox_wrt='canonicalAtlasSpace')

Prior structure/index map not found. Generating a new one.
File does not exist: /home/yuncong/MouseBrainAtlas/demo/demo_data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes/atlasV7_10.0um_scoreVolume_5N_L.bp
[Errno 2] No such file or directory: '/home/yuncong/MouseBrainAtlas/demo/demo_data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes/atlasV7_10.0um_scoreVolume_5N_L.bp'
Score volume for 5N_L does not exist.
File does not exist: /home/yuncong/MouseBrainAtlas/demo/demo_data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes/atlasV7_10.0um_scoreVolume_5N_R.bp
[Errno 2] No such file or directory: '/home/yuncong/MouseBrainAtlas/demo/demo_data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes/atlasV7_10.0um_scoreVolume_5N_R.bp'
Score volume for 5N_R does not exist.
File does not exist: /home/yuncong/MouseBrainAtlas/demo/demo_data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes/atlasV7_10.0um_scoreVolume_6N_L.bp
[Errno 2] No s

In [4]:
atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol = get_structure_centroids(vol_origin_dict=atlas_structures_wrt_canonicalAtlasSpace_atlasResol)

In [5]:
atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['12N'][:2], 0]
atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol = \
np.r_[atlas_structure_centroids_wrt_canonicalAtlasSpace_atlasResol['3N_R'][:2], 0]

In [6]:
# stack = 'CHATM2'
# stack = 'MD661'
stack = 'DEMO998'

In [7]:
intensity_volume_spec = dict(name=stack, resolution='10.0um', prep_id='wholebrainWithMargin', vol_type='intensity')

_, wholebrainWithMargin_origin_wrt_wholebrain_dataResol = \
DataManager.load_original_volume_v2(intensity_volume_spec, return_origin_instead_of_bbox=True)

wholebrainWithMargin_origin_wrt_wholebrain_dataResol_x, \
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol_y, \
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z = \
wholebrainWithMargin_origin_wrt_wholebrain_dataResol

In [8]:
# Pick the x/y coordinates on the intensity volume (no margin).
# Use the thumbnail sagittal panel (right bottom) in brain_labeling_gui. 
# Click a point while holding down space bar, then you can read off the coordinates at the GUI window's title bar.

# bug? Must click on lossless to get accurate coordinates.

# It seems subject_anchor2_wrt_wholebrainWithMargin_atlasResol are misnomer.
# they should really be subject_anchor1_wrt_wholebrain_atlasResol.

if stack == 'CHATM2':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([565,246,179]) * 20. / 10. # 12N
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([388,183,179]) * 20. / 10. # 3N_L
elif stack == 'CHATM3':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([740,258,230]) * 20. / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([550,195,230]) * 20. / 10.
elif stack == 'MD593':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([647*20., 233*20., 164*10.+138*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([474*20., 171*20., 164*10.+138*20.]) / 10.
elif stack == 'MD589':
    # Here 605 is the x of 12N's center, 260 is the y of 12N's center, 135 is the z of mid-sagittal plane. All in GUI's intensity volume voxel unit (20um/voxel).
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([605*20., 260*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+135*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([456*20., 175*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+135*20.]) / 10.
elif stack == 'MD585':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([614*20., 241*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+117*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([423*20., 165*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+145*20.]) / 10.
elif stack == 'MD590':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([650*20., 239*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+124*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([473*20., 159*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+124*20.]) / 10.
elif stack == 'MD591':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([637*20., 287*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+124*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([469*20., 190*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+124*20.]) / 10.
elif stack == 'MD592':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([647*20., 223*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+136*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([470*20., 180*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+136*20.]) / 10.
elif stack == 'MD594':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([650*20., 189*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+136*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([461*20., 151*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+136*20.]) / 10.
elif stack == 'MD595':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([674*20., 225*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+137*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([486*20., 180*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+137*20.]) / 10.
elif stack == 'MD598':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([628*20., 206*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+165*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([464*20., 164*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+165*20.]) / 10.
elif stack == 'MD599':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([682*20., 222*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+129*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([505*20., 175*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+129*20.]) / 10.
elif stack == 'MD602':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([668*20., 197*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+135*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([460*20., 157*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+135*20.]) / 10.
elif stack == 'MD603':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([651*20., 229*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+153*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([446*20., 168*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+153*20.]) / 10.
elif stack == 'MD661':
    subject_anchor1_wrt_wholebrainWithMargin_atlasResol = np.array([559*20., 209*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+140*20.]) / 10.
    subject_anchor2_wrt_wholebrainWithMargin_atlasResol = np.array([383*20., 166*20., wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10.+140*20.]) / 10.
elif stack == 'DEMO998':
    mid_z_wrt_wholebrainWithMargin = 0 # this must be changed
    subject_anchor1_wrt_wholebrain_atlasResol = np.array([wholebrainWithMargin_origin_wrt_wholebrain_dataResol_x * 10. + 558*20., 
                                                          wholebrainWithMargin_origin_wrt_wholebrain_dataResol_y * 10. + 207*20., 
                                                          wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10. + mid_z_wrt_wholebrainWithMargin*20.]) / 10.
    subject_anchor2_wrt_wholebrain_atlasResol = np.array([wholebrainWithMargin_origin_wrt_wholebrain_dataResol_x * 10. + 366*20., 
                                                          wholebrainWithMargin_origin_wrt_wholebrain_dataResol_y * 10. + 171*20., 
                                                          wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z*10. + mid_z_wrt_wholebrainWithMargin*20.]) / 10.
else:
    raise

In [16]:
def align_anchors(pm1, pm2, pf1, pf2):
    """
    Align the vector (pm1, pm2) and vector (pf1, pf2), and align the two middle points. 
    """
    
    t = ((pf1 + pf2) / 2. - (pm1 + pm2) / 2.)
    
    subject_d = pf1 - pf2
    atlas_d = pm1 - pm2

    subject_d_n = subject_d / np.linalg.norm(subject_d)
    atlas_d_n = atlas_d / np.linalg.norm(atlas_d)

    R = R_align_two_vectors(atlas_d_n, subject_d_n)
    
    T = np.zeros((3,4))
    T[:3, :3] = R
#     T[:3, 3] = np.dot(R, t)
    T[:3, 3] = t
    
    return T

In [17]:
if stack == 'DEMO998':
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol = \
    np.r_[wholebrainWithMargin_origin_wrt_wholebrain_dataResol_x,
          wholebrainWithMargin_origin_wrt_wholebrain_dataResol_y,
          wholebrainWithMargin_origin_wrt_wholebrain_dataResol_z]
else:
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol = \
np.r_[wholebrainWithMargin_origin_wrt_wholebrain_dataResol_x,
      wholebrainWithMargin_origin_wrt_wholebrain_dataResol_y,
      0]

    subject_anchor1_wrt_wholebrain_atlasResol = subject_anchor1_wrt_wholebrainWithMargin_atlasResol + \
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol # only valid because dataResol is same as atlasResol

    subject_anchor2_wrt_wholebrain_atlasResol = subject_anchor2_wrt_wholebrainWithMargin_atlasResol + \
    wholebrainWithMargin_origin_wrt_wholebrain_dataResol # only valid because dataResol is same as atlasResol

In [18]:
atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol

array([200.54678679,  68.66154597,   0.        ])

In [19]:
subject_anchor1_wrt_wholebrain_atlasResol

array([1233.,  547.,  448.])

In [111]:
T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = \
align_anchors(atlas_anchor1_wrt_canonicalAtlasSpace_atlasResol, atlas_anchor2_wrt_canonicalAtlasSpace_atlasResol,
             subject_anchor1_wrt_wholebrain_atlasResol, subject_anchor2_wrt_wholebrain_atlasResol)

In [112]:
print T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol

[[ 9.55516582e-01  2.94947156e-01  0.00000000e+00  1.00774282e+03]
 [-2.94947156e-01  9.55516582e-01  0.00000000e+00  5.30443355e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  4.48000000e+02]]


In [113]:
def quaternion_about_axis(angle, axis):
    """Return quaternion for rotation about axis.

    >>> q = quaternion_about_axis(0.123, [1, 0, 0])
    >>> numpy.allclose(q, [0.99810947, 0.06146124, 0, 0])
    True

    """
    q = np.array([0.0, axis[0], axis[1], axis[2]])
    qlen = np.linalg.norm(q)
    q *= np.sin(angle/2.0) / qlen
    q[0] = np.cos(angle/2.0)
    return q


def quaternion_matrix(quaternion):
    """Return homogeneous rotation matrix from quaternion.

    >>> M = quaternion_matrix([0.99810947, 0.06146124, 0, 0])
    >>> numpy.allclose(M, rotation_matrix(0.123, [1, 0, 0]))
    True
    >>> M = quaternion_matrix([1, 0, 0, 0])
    >>> numpy.allclose(M, numpy.identity(4))
    True
    >>> M = quaternion_matrix([0, 1, 0, 0])
    >>> numpy.allclose(M, numpy.diag([1, -1, -1, 1]))
    True

    """
    q = np.array(quaternion, dtype=np.float64, copy=True)
    n = np.dot(q, q)
    q *= np.sqrt(2.0 / n)
    q = np.outer(q, q)
    return np.array([
        [1.0-q[2, 2]-q[3, 3],     q[1, 2]-q[3, 0],     q[1, 3]+q[2, 0], 0.0],
        [    q[1, 2]+q[3, 0], 1.0-q[1, 1]-q[3, 3],     q[2, 3]-q[1, 0], 0.0],
        [    q[1, 3]-q[2, 0],     q[2, 3]+q[1, 0], 1.0-q[1, 1]-q[2, 2], 0.0],
        [                0.0,                 0.0,                 0.0, 1.0]])

rand_vec = np.random.uniform(size=3)
rand_vec = rand_vec / np.linalg.norm(rand_vec)
rand_rot_amount = np.random.uniform(high=np.deg2rad(5))
perturb_rot = quaternion_matrix(quaternion_about_axis(rand_rot_amount, rand_vec))

perturb_xyz = np.random.choice(np.arange(-30, 30), 3)

T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol = \
np.dot(T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol, perturb_rot)
T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol[:3, 3] += perturb_xyz

In [114]:
print T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol

[[ 9.63169856e-01  2.67574548e-01  2.67105543e-02  9.77742822e+02]
 [-2.66505671e-01  9.63095067e-01 -3.77939704e-02  5.38443355e+02]
 [-3.58373010e-02  2.92833300e-02  9.98928513e-01  4.68000000e+02]]


In [115]:
np.savetxt(os.path.join(DATA_ROOTDIR, 'CSHL_simple_global_registration', stack + '_T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol.txt'),
           T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol)

In [56]:
# bp.pack_ndarray_file(T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol, 
#                      os.path.join(DATA_ROOTDIR, 'CSHL_simple_global_registration', stack + '_T_atlas_wrt_canonicalAtlasSpace_subject_wrt_wholebrain_atlasResol.bp'))